In [1]:
# coding:u8
import json
import numpy as np
import pandas as pd
from tqdm import tqdm as tqdm
import gc,jieba
from gensim.models import Word2Vec
from sklearn.model_selection import KFold

In [29]:
# train_data = []
# train_data_dict = {}
# with open('data/raw_data/train.json','r') as f:
#     for line in f:
#         raw = json.loads(line)
#         train_data.append(raw)
#         train_data_dict[raw['text_id']] = raw
# # 读取知识库
# kb_data = []
# kb = {}
# with open('data/raw_data/kb_data','r') as f:
#     for line in f:
#         item = json.loads(line)
#         kb[item['subject_id']] = item
#         kb_data.append(item)


In [2]:
train_data = []
train_data_dict = {}
with open('data/raw_data/train.json', 'r') as f:
    for line in f:
        raw = eval(str(json.loads(line)).lower())
        train_data.append(raw)
        #train_data_dict[raw['text_id']] = raw
kb = {}
with open('data/raw_data/kb_data', 'r') as f:
    for line in f:
        item = eval(str(json.loads(line)).lower())
        kb[item['subject_id']] = item
# 补充别名实体进入kb
for s in train_data:
    for m in s['mention_data']:
        if m['kb_id'] == 'nil': continue
        if m['mention'] != kb[m['kb_id']]['subject'] and m['mention'] not in kb[m['kb_id']]['alias']:
            kb[m['kb_id']]['alias'] += [m['mention']]
            
name_id = {}
for kb_id in kb:
    for item in kb[kb_id]['alias']:
        if item not in name_id:
            name_id[item] = [kb_id]
        else:
            name_id[item].append(kb_id)
    if kb[kb_id]['subject'] not in name_id:
        name_id[kb[kb_id]['subject']] = [kb_id]
    else:
        name_id[kb[kb_id]['subject']].append(kb_id)
for id in name_id:
    name_id[id] = list(set(name_id[id]))

In [3]:
name_id['南京南站']

['311223', '130287']

In [4]:
"""
TODO:
预测type,
引入tfidf
"""

'\nTODO:\n预测type,\n引入tfidf\n'

In [5]:
train_data = train_data

In [6]:
# K = 0
# for s in tqdm(train_data):
#     mention_ner = s['mention_data']
#     for m in mention_ner:
#         if m['mention'] not in name_id:
#             continue
#         candidate_ids = name_id[m['mention']]
#         label = 0
#         for m_candidate_id in candidate_ids:
           
#             if m_candidate_id==m['kb_id']:
#                 label = 1
#                 K+=1
#         if label==0:
#             print(m)

In [14]:
def get_common_word(a, b):
        common_count = 0
        for word in a:
            if word in b:
                common_count += 1
        return common_count
def cal_similarity(params):
    index,sentence_a,sentence_b = params
    la = len(sentence_a)
    lb = len(sentence_b)

    char_a = [c for c in sentence_a]
    char_b = [c for c in sentence_b]
    word_a = jieba.lcut_for_search(sentence_a)
    word_b = jieba.lcut_for_search(sentence_b)
    bigram_a = [sentence_a[i:i + 2] for i in range(len(sentence_a) - 1)]
    bigram_b = [sentence_b[i:i + 2] for i in range(len(sentence_b) - 1)]
    trigram_a = [sentence_a[i:i + 3] for i in range(len(sentence_a) - 2)]
    trigram_b = [sentence_b[i:i + 3] for i in range(len(sentence_b) - 2)]

    common_word, common_word_score = 0, 0
    for w in word_b:
        if w in word_a:
            common_word += 1
            common_word_score += 1

    common_c = get_common_word(char_a, char_b)
    common_bigram = get_common_word(bigram_a, bigram_b)
    common_trigram = get_common_word(trigram_a, trigram_b)

    max_len = max(la, lb)
    min_len = min(la, lb)
    if max_len==0:
        max_len=0.01
    if min_len==0:
        min_len = 0.01
    numerical_feats = [max_len, min_len, common_word, common_word_score,
                                common_word / min_len, common_word / max_len,
                                common_word_score / min_len, common_word_score / max_len,
                                common_c / min_len, common_c / max_len,
                                common_bigram / min_len, common_bigram / max_len,
                                common_trigram / min_len, common_trigram / max_len,
                                common_word_score - common_word]
    return index, numerical_feats

In [9]:
# 提取candidate候选特征
kb_column = []
train_column = []
text_id = []
label_colum = []
num_attrs = []
num_abstract_words = []
num_alias = []
m_id = 0
m_id_list = []
num_candidate = []
equal_subject = []
shuminghao = [] 
entity_common = []
len_mention = []
mention_start = []
numerical_f = []
is_eng = []
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['object']

                if predicate['predicate'] == '摘要':
                    candidate_abstract = predicate['object']
                    candidate_abstract_numwords = len(candidate_abstract)
#                 if predicate['predicate'] == '标签':
#                     candidate_label += predicate['object']
            if m_candidate_id==m['kb_id']:
                label = 1
            else:
                label = 0
            train_column.append(m['mention'])
            kb_column.append(m_candidate_id)
            text_id.append(s['text_id'])
            label_colum.append(label)
            num_attrs.append(candidate_numattrs)
            num_abstract_words.append(candidate_abstract_numwords)
            num_alias.append(len(set(candidate_detail['alias']+[candidate_detail['subject']])))
            m_id_list.append(m_id)
            num_candidate.append(len(candidate_ids))
            equal_subject.append(candidate_detail['subject']==m['mention'])
            shuminghao.append('《'+candidate_detail['subject']+'》' in candidate_abstract)
#             if langid.classify(m['mention'])[0]=='en':
#                 len_mention.append(len(m['mention'].split()))
#                 is_eng.append(1)
#             else:
#                 len_mention.append(len(m['mention']))
#                 is_eng.append(0)
            len_mention.append(len(m['mention']))
            mention_start.append(m['offset'])
            
            i = 0
            for item in m_list:
                if item in candi_text: i+=1
            entity_common.append(i)
            #numerical_f.append(cal_similarity(candi_text,s['text']))
        m_id += 1
        
data = pd.DataFrame()
data['text_id'] = text_id
data['kb_id'] = kb_column
data['train_mention'] = train_column
data['label'] = label_colum
data['num_attrs'] = num_attrs
data['num_abstract_words'] = num_abstract_words
data['num_alias'] = num_alias
data['m_id'] = m_id_list
data['num_candidates'] = num_candidate 
data['mention_equal_subject'] = equal_subject
data['shuminghao'] = shuminghao
data['entity_common']  = entity_common
data['len_mention'] = len_mention
data['mention_start'] = list(map(int,mention_start))


100%|██████████| 90000/90000 [00:22<00:00, 4071.82it/s]

In [12]:
# 提取candidate候选特征
numerical_f = []
i = 0
for s in tqdm(train_data):
    mention_ner = s['mention_data']
    m_list = []
    for m in mention_ner:
        m_list.append(m['mention'])
    for m in mention_ner:
        if m['mention'] not in name_id : # 不能注销，因为有些实体没有or m['kb_id']=='nil':
            continue
        candidate_ids = name_id[m['mention']]
        for m_candidate_id in candidate_ids:
            # 统计配对特征
            candidate_numattrs = 0
            candidate_abstract_numwords = 0
            candidate_detail = kb[m_candidate_id]
            candi_text = ''
            for predicate in candidate_detail['data']:
                candidate_numattrs += 1
                candi_text += predicate['object']
                #candi_text += predicate['object']
            numerical_f.append((i,candi_text,s['text']))
            i+=1
            #numerical_f.append(cal_similarity(candi_text,s['text']))



100%|██████████| 90000/90000 [00:13<00:00, 6688.94it/s]

In [15]:
import multiprocessing
p = multiprocessing.Pool(7)
res = p.map(cal_similarity, numerical_f)
res = sorted(res,key=lambda x:x[0],reverse=False)

In [17]:
nume_f = [x[1] for x in res]

In [19]:
numerical_f = np.array(nume_f)
for i in range(numerical_f.shape[1]):
    data['numerical_%d'%i] = numerical_f[:,i]

In [20]:
data.head()

,text_id,kb_id,train_mention,label,num_attrs,num_abstract_words,num_alias,m_id,num_candidates,mention_equal_subject,...,numerical_5,numerical_6,numerical_7,numerical_8,numerical_9,numerical_10,numerical_11,numerical_12,numerical_13,numerical_14
0,1,311223,南京南站,1,17,447,3,0,2,True,...,0.012638,0.421053,0.012638,6.473684,0.194313,2.473684,0.074250,1.052632,0.031596,0.0
1,1,130287,南京南站,0,9,338,2,0,2,True,...,0.018648,0.421053,0.018648,4.210526,0.186480,1.736842,0.076923,0.842105,0.037296,0.0
2,1,341096,高铁,1,10,239,1,1,2,True,...,0.009434,0.157895,0.009434,1.473684,0.088050,0.315789,0.018868,0.000000,0.000000,0.0
3,1,213561,高铁,0,1,0,2,1,2,False,...,0.000000,0.000000,0.000000,0.500000,0.105263,0.000000,0.000000,0.000000,0.000000,0.0
4,1,311223,南京南站,1,17,447,3,2,2,True,...,0.012638,0.421053,0.012638,6.473684,0.194313,2.473684,0.074250,1.052632,0.031596,0.0


In [22]:
f_columns = ['numerical_%d'%i for i in range(15)]

In [23]:
data[f_columns].to_pickle('data/numerical_f.pkl')

In [9]:
# f_nume_df = pd.read_pickle('data/numerical_f.pkl')
# for item in f_columns:
#     data[item] = f_nume_df[item]

In [3]:
data = pd.read_pickle('data/step1.pkl')

In [24]:
data.to_pickle('data/step1.pkl')

In [4]:
# 引入embedding特征
mention_vector = np.load('entity_embedding/gensim_vector.npy')
entity_embedding = Word2Vec.load('embedding/w2v.model')
from sklearn.metrics import mean_squared_error as mse

/home/zhukaihua/anaconda3/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [5]:
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
mention_vector.shape

(1834968, 100)

In [7]:
import multiprocessing

def cal_distance(params) :
    index,left,right = params
#     left = entity_embedding.wv.word_vec(left)
#     right = mention_vector(right)
    return index,mse(left,right),cosine_similarity(left.reshape(1,-1),right.reshape(1,-1))
  

In [8]:
numlist = []
for index,row in data.iterrows():
    numlist.append((index,entity_embedding.wv.word_vec(row['kb_id']),mention_vector[index]))

In [9]:
p = multiprocessing.Pool(6)
res = p.map(cal_distance, numlist)

In [10]:
res = sorted(res,key=lambda x:x[0],reverse=False)

In [11]:
data['cos_distance'] = [x[1] for x in res]
data['mse_distance'] = [x[2] for x in res]

In [12]:
data['mse_distance'] = data['mse_distance'].apply(lambda x:x[0][0])

In [13]:
del res,numlist
gc.collect()

87

In [ ]:
del entity_embedding,mention_vector,numerical_f
gc.collect()

In [14]:
# candidate　type的one-hot特征
data['type'] = data.apply(lambda x:kb[x['kb_id']]['type'][0],axis=1)
#data = pd.concat([data,pd.get_dummies(data['type'])],axis=1)

In [15]:
# target encoding
data_groupbytype = data.groupby('type',as_index=False)['label'].agg({
    'type_label_mean':'mean','type_label_count':'count'})
data = data.merge(data_groupbytype,on='type',how='left')

In [16]:
# 流行度特征
# 总流行度
data_ctr = []
kfold = KFold(n_splits=9,shuffle=False,random_state=2019)
for train_index,test_index in kfold.split(data):
    print(test_index)
    stat_train_data,stat_test_data = data.loc[train_index,:],data.loc[test_index,:]
    #print(test_data.head())
    train_group = stat_train_data.groupby('kb_id',as_index=False)['label'].agg({'label_mean':'mean','label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on='kb_id',how='left')
    print(stat_test_data[stat_test_data.label_mean.isnull()].shape)
    train_group = stat_train_data.groupby(['kb_id','train_mention'])['label'].agg({'m_label_mean':'mean','m_label_count':'count'})
    stat_test_data = stat_test_data.merge(train_group,on=['kb_id','train_mention'],how='left')
    data_ctr.append(stat_test_data)
data_ctr = pd.concat(data_ctr,axis=0)


[     0      1      2 ... 203883 203884 203885]
(20029, 36)


/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  if sys.path[0] == '':


[203886 203887 203888 ... 407769 407770 407771]
(19987, 36)
[407772 407773 407774 ... 611655 611656 611657]
(19324, 36)
[611658 611659 611660 ... 815540 815541 815542]
(20262, 36)
[ 815543  815544  815545 ... 1019425 1019426 1019427]
(19479, 36)
[1019428 1019429 1019430 ... 1223310 1223311 1223312]
(19668, 36)
[1223313 1223314 1223315 ... 1427195 1427196 1427197]
(19561, 36)
[1427198 1427199 1427200 ... 1631080 1631081 1631082]
(19584, 36)
[1631083 1631084 1631085 ... 1834965 1834966 1834967]
(19649, 36)


In [17]:
# rank 特征
to_rank = ['num_attrs', 'num_abstract_words', 'num_alias', 'label_mean', 'label_count', 'm_label_mean', 
             'm_label_count','num_candidates','mention_equal_subject','shuminghao','entity_common',
           'cos_distance','mse_distance','type_label_mean','numerical_0', 'numerical_1', 'numerical_2',
       'numerical_3', 'numerical_4', 'numerical_5', 'numerical_6',
       'numerical_7', 'numerical_8', 'numerical_9', 'numerical_10',
       'numerical_11', 'numerical_12', 'numerical_13', 'numerical_14'
          ]
for f in to_rank:
    data_group = data_ctr.groupby('m_id')[f].rank(ascending=False)
    data_ctr['rank_'+f] = data_group
    try:
        data_ctr['rank_'+f] = data_ctr['rank_'+f].astype('int32')
    except:
        pass

In [18]:
for item in data_ctr.columns:
    if data_ctr.dtypes[item]=='float64':
        print(item)
        data_ctr[item] = data_ctr[item].astype('float32')

numerical_0
numerical_1
numerical_2
numerical_3
numerical_4
numerical_5
numerical_6
numerical_7
numerical_8
numerical_9
numerical_10
numerical_11
numerical_12
numerical_13
numerical_14
cos_distance
mse_distance
type_label_mean
label_mean
label_count
m_label_mean
m_label_count
rank_label_mean
rank_label_count
rank_m_label_mean
rank_m_label_count


In [19]:
del data
gc.collect()

187

In [20]:
for item in data_ctr.columns:
    print(item)

text_id
kb_id
train_mention
label
num_attrs
num_abstract_words
num_alias
m_id
num_candidates
mention_equal_subject
shuminghao
entity_common
len_mention
mention_start
numerical_0
numerical_1
numerical_2
numerical_3
numerical_4
numerical_5
numerical_6
numerical_7
numerical_8
numerical_9
numerical_10
numerical_11
numerical_12
numerical_13
numerical_14
cos_distance
mse_distance
type
type_label_mean
type_label_count
label_mean
label_count
m_label_mean
m_label_count
rank_num_attrs
rank_num_abstract_words
rank_num_alias
rank_label_mean
rank_label_count
rank_m_label_mean
rank_m_label_count
rank_num_candidates
rank_mention_equal_subject
rank_shuminghao
rank_entity_common
rank_cos_distance
rank_mse_distance
rank_type_label_mean
rank_numerical_0
rank_numerical_1
rank_numerical_2
rank_numerical_3
rank_numerical_4
rank_numerical_5
rank_numerical_6
rank_numerical_7
rank_numerical_8
rank_numerical_9
rank_numerical_10
rank_numerical_11
rank_numerical_12
rank_numerical_13
rank_numerical_14


In [21]:
# mean and var
to_rank = ['num_attrs', 'num_abstract_words', 'num_alias', 'label_mean', 'label_count', 'm_label_mean', 
             'm_label_count','num_candidates','mention_equal_subject','shuminghao','entity_common',
           'cos_distance','mse_distance','type_label_mean','numerical_0', 'numerical_1', 'numerical_2',
       'numerical_3', 'numerical_4', 'numerical_5', 'numerical_6',
       'numerical_7', 'numerical_8', 'numerical_9', 'numerical_10',
       'numerical_11', 'numerical_12', 'numerical_13', 'numerical_14'
          ]
for f in tqdm(to_rank):
    data_group = data_ctr.groupby('m_id')[f].agg({f+'_var':'var',f+'_mean':'mean',f+'_max':'max',f+'_min':'min'})
    data_ctr = data_ctr.merge(data_group,on='m_id',how='left')

  0%|          | 0/29 [00:00<?, ?it/s]/home/zhukaihua/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  # Remove the CWD from sys.path while we load stuff.
100%|██████████| 29/29 [02:18<00:00,  6.69s/it]


In [22]:
list(data_ctr.columns)

['text_id',
 'kb_id',
 'train_mention',
 'label',
 'num_attrs',
 'num_abstract_words',
 'num_alias',
 'm_id',
 'num_candidates',
 'mention_equal_subject',
 'shuminghao',
 'entity_common',
 'len_mention',
 'mention_start',
 'numerical_0',
 'numerical_1',
 'numerical_2',
 'numerical_3',
 'numerical_4',
 'numerical_5',
 'numerical_6',
 'numerical_7',
 'numerical_8',
 'numerical_9',
 'numerical_10',
 'numerical_11',
 'numerical_12',
 'numerical_13',
 'numerical_14',
 'cos_distance',
 'mse_distance',
 'type',
 'type_label_mean',
 'type_label_count',
 'label_mean',
 'label_count',
 'm_label_mean',
 'm_label_count',
 'rank_num_attrs',
 'rank_num_abstract_words',
 'rank_num_alias',
 'rank_label_mean',
 'rank_label_count',
 'rank_m_label_mean',
 'rank_m_label_count',
 'rank_num_candidates',
 'rank_mention_equal_subject',
 'rank_shuminghao',
 'rank_entity_common',
 'rank_cos_distance',
 'rank_mse_distance',
 'rank_type_label_mean',
 'rank_numerical_0',
 'rank_numerical_1',
 'rank_numerical_2',
 

In [23]:
data_ctr.reset_index(drop=True,inplace=True)
data_ctr.to_pickle('features/1_stat_feature.pkl')